In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline\\steps'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig_LR:
    root_dir: Path
    X_train_data_path : Path
    y_train_data_path : Path
    model_name : str
    model_dir_name : str
    runtime_name : str

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories,save_json

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config_LR(self) -> ModelTrainerConfig_LR:
        config = self.config.model_trainer_LR
        
        create_directories([config.root_dir])
        create_directories([os.path.join(config.root_dir,config.model_dir_name)])
        model_trainer_config = ModelTrainerConfig_LR(
            root_dir = config.root_dir,
            X_train_data_path=config.X_train_data_path,
            y_train_data_path=config.y_train_data_path,
            model_name = config.model_name,
            model_dir_name = config.model_dir_name,
            runtime_name=config.runtime_name
        )
        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.mlProject.utils import logger
from sklearn.linear_model import LinearRegression
import joblib
import numpy as np
import time

In [9]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig_LR):
        self.config = config
        
    def load_data(self):
        X_train =np.array(pd.read_csv(self.config.X_train_data_path))
        y_train =np.array(pd.read_csv(self.config.y_train_data_path))
        
        return X_train,y_train
        
    def train(self,X_train,y_train):    
        lr = LinearRegression()
        start_lr = time.time()
        lr.fit(X_train, y_train)
        end_lr = time.time()
        
        joblib.dump(lr,os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.model_name))
        training_time = end_lr - start_lr
        training_time = {"training time" : training_time}
        save_json(path=Path(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.runtime_name)),data=training_time)        




In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config_LR()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    X_train_scaled,y_train_scaled = model_trainer_config.load_data()
    model_trainer_config.train(X_train_scaled,y_train_scaled)
except Exception as e:
    raise e

[2024-04-17 15:40:40,179: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-17 15:40:40,183: INFO: common: created directory at: artifacts]
[2024-04-17 15:40:40,188: INFO: common: created directory at: artifacts/model_trainer]
[2024-04-17 15:40:40,192: INFO: common: created directory at: artifacts/model_trainer\LR]
[2024-04-17 15:40:40,598: INFO: common: json file saved at: artifacts\model_trainer\LR\lr_runtime.json]
